In [1]:
import ipyparallel as ipp
c = ipp.Client(profile='remote', cluster_id='remote')
dview = c[:]
print(c.ids)

[0, 1, 2, 3, 4, 5]


In [2]:
%px import numpy as np
%px import time
%px from mpi4py import MPI

# 初始化MPI環境
%px comm = MPI.COMM_WORLD
%px rank = comm.Get_rank()
%px size = comm.Get_size()


In [3]:
dview.run('D://code/workplace_py/Parallel/PythonCode/rank_mpi.py')
%px totalhost = hostname()
dview['totalhost']

['LENOVO', 'LENOVO', 'LENOVO', 'LENOVO', 'LAPTOP-NBMNPREL', 'LAPTOP-NBMNPREL']

In [4]:
%%px
print(comm.rank)

[stdout:0] 0
[stdout:1] 1
[stdout:2] 2
[stdout:3] 3
[stdout:4] 1
[stdout:5] 0


In [5]:
%%px
# 定義Bootstrap
def bootstrap(data, n_bootstrap):
    n = len(data)
    bootstrap_samples = np.zeros((n_bootstrap, n))
    for i in range(n_bootstrap):
        bootstrap_sample = np.random.choice(data, size=n, replace=True)
        bootstrap_samples[i] = bootstrap_sample
    return bootstrap_samples

In [7]:
%%px

a = time.time()
# 假設有一組原始數據
strong_hand = np.array([77.3, 95.1, 97.2, 94.5, 83.6, 90.2, 84.3])
weak_hand = np.array([74.6, 83.4, 80.6, 90.2, 78.7, 83.6, 76.2])

#計算原始的平均數差異
u_diff = np.mean(strong_hand) - np.mean(weak_hand)

# 將10個bootstrap樣本平均分配給每個CPU
n_bootstrap = int(1000000/2)
n_bootstrap_per_cpu = n_bootstrap // size

# 每個CPU執行自己分配到的bootstrap樣本
bootstrap_samples_local1 = bootstrap(strong_hand, n_bootstrap_per_cpu)
bootstrap_samples_local2 = bootstrap(weak_hand, n_bootstrap_per_cpu)

# 將每個CPU的結果收集到根節點
bootstrap_samples_all1= comm.gather(bootstrap_samples_local1, root=0)
bootstrap_samples_all2 = comm.gather(bootstrap_samples_local2, root=0)

if rank == 0:

    # 計算每個bootstrap樣本的平均數
    bootstrap_means1 = np.mean(bootstrap_samples_all1, axis=(0, 2))
    bootstrap_means2 = np.mean(bootstrap_samples_all2, axis=(0, 2))
    mean_diff = bootstrap_means1 - bootstrap_means2
    
# 結束時間
b = time.time()
print(f"time {b-a:.3f}")

[stdout:0] time 4.666
[stdout:1] time 4.634
[stdout:2] time 4.636
[stdout:3] time 4.638
[stdout:4] time 15.813
[stdout:5] time 15.868
